In [1]:
import pandas as pd
import numpy as np
import os
import datetime as dt

# Importing sector DFs

In [2]:
# importing actual energy generation
PATH_TO_DATA = '/Users/R/code/HaukeFock/predict_energy_generation/raw_data/'
companies = !ls /Users/R/code/HaukeFock/predict_energy_generation/raw_data/energy_generation/

## Actual Energy Generation

In [3]:
columns = ['Date', 'Time of day', 'Biomass[MWh]', 'Hydropower[MWh]',
       'Wind offshore[MWh]', 'Wind onshore[MWh]', 'Photovoltaics[MWh]',
       'Other renewable[MWh]', 'Nuclear[MWh]', 'Fossil brown coal[MWh]',
       'Fossil hard coal[MWh]', 'Fossil gas[MWh]', 'Hydro pumped storage[MWh]',
       'Other conventional[MWh]']
dataframes = {}

for company in companies:
    #print(company)
    exec(f"df_{company} = pd.DataFrame(columns=columns)")
    data_paths = !ls /Users/R/code/HaukeFock/predict_energy_generation/raw_data/energy_generation/{company}
    for file in data_paths:
        #print(file)
        data = pd.read_csv(f'{PATH_TO_DATA}/energy_generation/{company}/{file}', sep=';')
        exec(f"df_{company} = df_{company}.append(data, ignore_index=True)")
    exec(f"dataframes['df_{company}'] = df_{company}")
    print(f'df_{company} Loaded!!!')

/Users/R/.pyenv/versions/3.8.6/envs/predict_energy_generation/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Users/R/.pyenv/versions/3.8.6/envs/predict_energy_generation/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (4,11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


df_50hertz Loaded!!!


/Users/R/.pyenv/versions/3.8.6/envs/predict_energy_generation/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (2,3,4,6,9,10,12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Users/R/.pyenv/versions/3.8.6/envs/predict_energy_generation/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (2,3,5,6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Users/R/.pyenv/versions/3.8.6/envs/predict_energy_generation/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (5,12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


df_amprion Loaded!!!
df_tennet Loaded!!!


/Users/R/.pyenv/versions/3.8.6/envs/predict_energy_generation/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Users/R/.pyenv/versions/3.8.6/envs/predict_energy_generation/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (5,11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


df_transnet_bw Loaded!!!


/Users/R/.pyenv/versions/3.8.6/envs/predict_energy_generation/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
# dropping unecessary columns 
df_50hertz['Date'] = df_50hertz['Date'][:140448].append(df_50hertz['Datum'][140448:])
df_50hertz['Time of day'] = df_50hertz['Time of day'][:140448].append(df_50hertz['Uhrzeit'][140448:])

df_tennet['Date'] = df_tennet['Datum'][:70272].append(df_tennet['Date'][70272:])
df_tennet['Time of day'] = df_tennet['Uhrzeit'][:70272].append(df_tennet['Time of day'][70272:])

df_50hertz.drop(columns=['Datum', 'Uhrzeit'], inplace=True)
df_tennet.drop(columns=['Datum', 'Uhrzeit'], inplace=True)

# missing values resume for each area
for key, df in dataframes.items():
    if (df.columns != columns).all():
        print(f'{key} has different columns than expected!')
    else:
        print(f'>>>>>>>>>>>>>> {key} <<<<<<<<<<<<<<\n')
        print(f'Shape: {df.shape}\n')
        print(f'NaN values: \n{df.isna().sum()}\n')

>>>>>>>>>>>>>> df_50hertz <<<<<<<<<<<<<<

Shape: (210720, 14)

NaN values: 
Date                              0
Time of day                       0
Biomass[MWh]                      0
Hydropower[MWh]                   0
Wind offshore[MWh]                0
Wind onshore[MWh]                 0
Photovoltaics[MWh]                0
Other renewable[MWh]              0
Nuclear[MWh]                 210720
Fossil brown coal[MWh]            0
Fossil hard coal[MWh]             0
Fossil gas[MWh]                   0
Hydro pumped storage[MWh]         0
Other conventional[MWh]           0
dtype: int64

>>>>>>>>>>>>>> df_amprion <<<<<<<<<<<<<<

Shape: (210720, 14)

NaN values: 
Date                              0
Time of day                       0
Biomass[MWh]                      0
Hydropower[MWh]                   0
Wind offshore[MWh]           210720
Wind onshore[MWh]                 0
Photovoltaics[MWh]                0
Other renewable[MWh]              0
Nuclear[MWh]                      0
Fossil

In [5]:
numeric_values = ['Biomass[MWh]',
                 'Hydropower[MWh]',
                 'Wind offshore[MWh]',
                 'Wind onshore[MWh]',
                 'Photovoltaics[MWh]',
                 'Other renewable[MWh]',
                 'Nuclear[MWh]',
                 'Fossil brown coal[MWh]',
                 'Fossil hard coal[MWh]',
                 'Fossil gas[MWh]',
                 'Hydro pumped storage[MWh]',
                 'Other conventional[MWh]']

def set_to_numeric(dataframes, numeric_values):
    # sets the features to numerical values
    for column in numeric_values:
        #print(column)
        for key, df in dataframes.items():
            #print(f'{key} => {column}')
            df[column] = df[column].apply(lambda x: str(x).replace(',',''))
            df[column] = pd.to_numeric(df[column], errors='coerce')

set_to_numeric(dataframes, numeric_values)

In [6]:
# join 'Date' and 'Time of Day' into one
def set_datetime(dataframes):
    for df in dataframes.keys():
        print(f'Working on {df}')
        exec(f"{df}['Date'] = pd.to_datetime({df}['Date']).apply(lambda x: x.strftime('%Y-%m-%d '))")
        exec(f"{df}['Time of day'] = pd.to_datetime({df}['Time of day']).apply(lambda x: x.strftime('%H:%M'))")
        exec(f"{df}['Date'] = pd.to_datetime({df}['Date'] + {df}['Time of day'])")
        exec(f"{df}.drop(columns=['Time of day'], inplace=True)")
set_datetime(dataframes)

Working on df_50hertz
Working on df_amprion
Working on df_tennet
Working on df_transnet_bw


In [8]:
df_50hertz.tail()

,Date,Biomass[MWh],Hydropower[MWh],Wind offshore[MWh],Wind onshore[MWh],Photovoltaics[MWh],Other renewable[MWh],Nuclear[MWh],Fossil brown coal[MWh],Fossil hard coal[MWh],Fossil gas[MWh],Hydro pumped storage[MWh],Other conventional[MWh]
210715,2021-01-01 22:45:00,218.0,26.0,31.0,423.0,0.0,18.0,NaN,1547.0,259.0,162.0,46.0,235.0
210716,2021-01-01 23:00:00,219.0,26.0,37.0,409.0,0.0,18.0,NaN,1556.0,256.0,159.0,32.0,232.0
210717,2021-01-01 23:15:00,215.0,25.0,34.0,400.0,0.0,18.0,NaN,1545.0,245.0,158.0,15.0,228.0
210718,2021-01-01 23:30:00,215.0,25.0,29.0,416.0,0.0,18.0,NaN,1579.0,236.0,158.0,28.0,231.0
210719,2021-01-01 23:45:00,219.0,26.0,24.0,429.0,0.0,18.0,NaN,1561.0,226.0,155.0,18.0,234.0


## Forcasted Energy generation

In [7]:
# importing actual energy generation
forecasted_columns = ['Date', 'Time of day', 'Total[MWh]', 'Wind offshore[MWh]', 
           'Wind onshore[MWh]', 'Photovoltaics[MWh]', 'Other[MWh]']
forecasted_dataframes = {}

for company in companies:
    #print(company)
    exec(f"df_{company}_forecasted = pd.DataFrame(columns=forecasted_columns)")
    data_paths = !ls /Users/R/code/HaukeFock/predict_energy_generation/raw_data/forcasted_generation/{company}
    for file in data_paths:
        #print(file)
        data = pd.read_csv(f'{PATH_TO_DATA}/forcasted_generation/{company}/{file}', sep=';')
        exec(f"df_{company}_forecasted = df_{company}_forecasted.append(data, ignore_index=True)")
    exec(f"forecasted_dataframes['df_{company}_forecasted'] = df_{company}_forecasted")
    print(f'df_{company}_forecasted Loaded!!!')

df_50hertz_forecasted Loaded!!!
df_amprion_forecasted Loaded!!!
df_tennet_forecasted Loaded!!!
df_transnet_bw_forecasted Loaded!!!


In [8]:
# dropping unecessary columns
df_tennet_forecasted['Date'] = df_tennet_forecasted.Date[:210720].append(df_tennet_forecasted.Datum[210720:222236])\
                                                                .append(df_tennet_forecasted.Date[222236:])


df_tennet_forecasted['Time of day'] = df_tennet_forecasted['Time of day'][:210720].append(df_tennet_forecasted.Uhrzeit[210720:222236])\
                                                          .append(df_tennet_forecasted['Time of day'][222236:]).isna().sum()
df_tennet_forecasted.drop(columns=['Datum', 'Uhrzeit'], inplace=True)

# missing values resume for each area
for key, df in forecasted_dataframes.items():
    if (df.columns != forecasted_columns).all():
        print(f'{key} has different columns than expected!')
    else:
        print(f'>>>>>>>>>>>>>> {key} <<<<<<<<<<<<<<\n')
        print(f'Shape: {df.shape}\n')
        print(f'NaN values: \n{df.isna().sum()}\n')

>>>>>>>>>>>>>> df_50hertz_forecasted <<<<<<<<<<<<<<

Shape: (223388, 7)

NaN values: 
Date                  0
Time of day           0
Total[MWh]            0
Wind offshore[MWh]    0
Wind onshore[MWh]     0
Photovoltaics[MWh]    0
Other[MWh]            0
dtype: int64

>>>>>>>>>>>>>> df_amprion_forecasted <<<<<<<<<<<<<<

Shape: (223388, 7)

NaN values: 
Date                       0
Time of day                0
Total[MWh]                 0
Wind offshore[MWh]    223388
Wind onshore[MWh]          0
Photovoltaics[MWh]         0
Other[MWh]                 0
dtype: int64

>>>>>>>>>>>>>> df_tennet_forecasted <<<<<<<<<<<<<<

Shape: (223388, 7)

NaN values: 
Date                  0
Time of day           0
Total[MWh]            0
Wind offshore[MWh]    0
Wind onshore[MWh]     0
Photovoltaics[MWh]    0
Other[MWh]            0
dtype: int64

>>>>>>>>>>>>>> df_transnet_bw_forecasted <<<<<<<<<<<<<<

Shape: (223388, 7)

NaN values: 
Date                       0
Time of day                0
Total[MWh]    

In [9]:
# setting columns to numeric
forecasted_numeric_columns = ['Total[MWh]','Wind offshore[MWh]','Wind onshore[MWh]',
                            'Photovoltaics[MWh]','Other[MWh]']

set_to_numeric(forecasted_dataframes, forecasted_numeric_columns)

# join 'Date' and 'Time of Day' into one
set_datetime(forecasted_dataframes)

Working on df_50hertz_forecasted
Working on df_amprion_forecasted
Working on df_tennet_forecasted
Working on df_transnet_bw_forecasted


## Production capacity

In [229]:
capacity_columns = ['Date', 'Time of day', 'Biomass[MW]', 'Hydropower[MW]',
       'Wind offshore[MW]', 'Wind onshore[MW]', 'Photovoltaics[MW]',
       'Other renewable[MW]', 'Nuclear[MW]', 'Fossil brown coal[MW]',
       'Fossil hard coal[MW]', 'Fossil gas[MW]', 'Hydro pumped storage[MW]',
       'Other conventional[MW]']
capacity_dataframes = {}

for company in companies:
    #print(company)
    exec(f"df_{company}_capacity = pd.DataFrame(columns=capacity_columns)")
    data_paths = !ls /Users/R/code/HaukeFock/predict_energy_generation/raw_data/energy_capacity/{company}
    for file in data_paths:
        #print(file)
        data = pd.read_csv(f'{PATH_TO_DATA}/energy_capacity/{company}/{file}', sep=';')
        exec(f"df_{company}_capacity = df_{company}_capacity.append(data, ignore_index=True)")
    exec(f"capacity_dataframes['df_{company}_capacity'] = df_{company}_capacity")
    print(f'df_{company}_capacity Loaded!!!')

df_50hertz_capacity Loaded!!!
df_amprion_capacity Loaded!!!
df_tennet_capacity Loaded!!!
df_transnet_bw_capacity Loaded!!!


In [230]:
# dropping duplicates
for df in capacity_dataframes.keys():
    exec(f"{df} = {df}.drop(index=[2,5,8]).reset_index(drop=True)")

In [231]:
capacity_numeric_values = ['Biomass[MW]', 'Hydropower[MW]', 'Wind offshore[MW]', 'Wind onshore[MW]',
                           'Photovoltaics[MW]', 'Other renewable[MW]', 'Nuclear[MW]', 'Fossil brown coal[MW]',
                           'Fossil hard coal[MW]', 'Fossil gas[MW]', 'Hydro pumped storage[MW]', 
                           'Other conventional[MW]']

set_to_numeric(capacity_dataframes, capacity_numeric_values)

In [232]:
set_datetime(capacity_dataframes)

Working on df_50hertz_capacity
Working on df_amprion_capacity
Working on df_tennet_capacity
Working on df_transnet_bw_capacity


# Group data

## Daily

In [217]:
#group energy generation per day 
for df in dataframes.keys():
    exec(f"{df} = {df}.groupby(by={df}['Date'].dt.date).sum().reset_index()")

In [218]:
#group forcasted energy generation per day 
for df in forecasted_dataframes.keys():
    exec(f"{df} = {df}.groupby(by={df}['Date'].dt.date).sum().reset_index()")

## Hourly

In [14]:
#group energy generation per hour 
for df in dataframes.keys():
    exec(f"{df}.index = {df}.Date")
    exec(f"{df} = {df}.resample('h').sum().reset_index()")

In [15]:
#group forcasted energy generation per hour 
for df in forecasted_dataframes.keys():
    exec(f"{df}.index = {df}.Date")
    exec(f"{df} = {df}.resample('h').sum().reset_index()")

# Creating complete DF

## Actual

In [16]:
df_deutschland = pd.DataFrame()

for df, company in zip(dataframes.keys(), companies):
        exec(f"{df}['region'] = company")
        exec(f"df_deutschland = df_deutschland.append({df}, ignore_index=True)")

df_deutschland.shape

(210528, 14)

## Forcasted

In [17]:
df_deutschland_forecasted = pd.DataFrame()

for df, company in zip(forecasted_dataframes.keys(), companies):
        exec(f"{df}['region'] = company")
        exec(f"df_deutschland_forecasted = df_deutschland_forecasted.append({df}, ignore_index=True)")

df_deutschland_forecasted.shape

(224137, 7)

## Capacity

In [207]:
df_deutschland_capacity = pd.DataFrame()

for df, company in zip(capacity_dataframes.keys(), companies):
        exec(f"{df}['region'] = company")
        exec(f"df_deutschland_capacity = df_deutschland_capacity.append({df}, ignore_index=True)")

df_deutschland_capacity.shape

(28, 14)

# Saving DFs

## Daily

### Actual

In [221]:
final_columns = ['Date', 'Wind offshore[MWh]', 'Wind onshore[MWh]', 'Photovoltaics[MWh]', 'region']

dataframes['df_deutschland'] = df_deutschland

for df in dataframes.keys():
    exec(f"{df}[final_columns].to_csv('/Users/R/code/HaukeFock/predict_energy_generation/raw_data/daily/energy_generation_data/{df}.csv', index=False)")

In [223]:
df = pd.read_csv('/Users/R/code/HaukeFock/predict_energy_generation/raw_data/daily/energy_generation_data/df_50hertz.csv')
df.head()

,Date,Wind offshore[MWh],Wind onshore[MWh],Photovoltaics[MWh],region
0,2015-01-01,287.75,30682.25,1812.75,50hertz
1,2015-01-02,238.75,61332.25,424.25,50hertz
2,2015-01-03,270.75,55415.00,656.50,50hertz
3,2015-01-04,272.50,49897.00,1194.00,50hertz
4,2015-01-05,98.50,32000.25,484.50,50hertz


### Forecasted

In [224]:
final_columns = ['Date', 'Wind offshore[MWh]', 'Wind onshore[MWh]', 'Photovoltaics[MWh]', 'region']

forecasted_dataframes['df_deutschland_forecasted'] = df_deutschland_forecasted

for df in forecasted_dataframes.keys():
    exec(f"{df}[final_columns].to_csv('/Users/R/code/HaukeFock/predict_energy_generation/raw_data/daily/forecasted_generation_data/{df}.csv', index=False)")

### Capacity

In [225]:
final_columns = ['Date', 'Wind offshore[MW]', 'Wind onshore[MW]', 'Photovoltaics[MW]', 'region']

capacity_dataframes['df_deutschland_capacity'] = df_deutschland_capacity

for df in capacity_dataframes.keys():
    exec(f"{df}[final_columns].to_csv('/Users/R/code/HaukeFock/predict_energy_generation/raw_data/daily/energy_capacity_data/{df}.csv', index=False)")
    

KeyError: "['region'] not in index"

## Hourly

### Actual

In [245]:
final_columns = ['Date', 'Wind offshore[MWh]', 'Wind onshore[MWh]', 'Photovoltaics[MWh]', 'region']

dataframes['df_deutschland'] = df_deutschland

for df in dataframes.keys():
    exec(f"{df}[final_columns].to_csv('/Users/R/code/HaukeFock/predict_energy_generation/raw_data/hourly/energy_generation_data/{df}.csv', index=False)")
    

### Forecasted

In [246]:
final_columns = ['Date', 'Wind offshore[MWh]', 'Wind onshore[MWh]', 'Photovoltaics[MWh]', 'region']

forecasted_dataframes['df_deutschland_forecasted'] = df_deutschland_forecasted

for df in forecasted_dataframes.keys():
    exec(f"{df}[final_columns].to_csv('/Users/R/code/HaukeFock/predict_energy_generation/raw_data/hourly/forecasted_generation_data/{df}.csv', index=False)")

# scratch

In [148]:
df_deutschland.groupby(by=df_deutschland['Date'].dt.date).sum()

AttributeError: Can only use .dt accessor with datetimelike values

In [150]:
pd.to_datetime(df_deutschland['Date'])

0      2015-01-01
1      2015-01-02
2      2015-01-03
3      2015-01-04
4      2015-01-05
          ...    
8767   2020-12-28
8768   2020-12-29
8769   2020-12-30
8770   2020-12-31
8771   2021-01-01
Name: Date, Length: 8772, dtype: datetime64[ns]